# Import ERA-5 Climate data and filter by trap locations to reduce size

In [1]:
import numpy as np
from datetime import datetime

In [2]:
import pandas as pd

In [3]:
import xarray as xr
import flox

/Users/katherinedixon/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
state = "PNW"
state_code = "PNW"

In [6]:
f80 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_1980-1989.nc", decode_times = True)
f90 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_1990-1999.nc", decode_times = True)
f00 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_2000-2009.nc", decode_times = True)
f10 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_2010-2020.nc", decode_times = True)
f19 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_2019.nc", decode_times = True)


In [5]:
f23 = xr.open_dataset("/Volumes/My Book/Climate/ERA/"+state_code+"_ERA-5_2021-2023.nc", decode_times = True)


In [8]:
print(f80.longitude)
print(f80.latitude)

<xarray.DataArray 'longitude' (longitude: 25)>
array([-122.  , -121.75, -121.5 , -121.25, -121.  , -120.75, -120.5 , -120.25,
       -120.  , -119.75, -119.5 , -119.25, -119.  , -118.75, -118.5 , -118.25,
       -118.  , -117.75, -117.5 , -117.25, -117.  , -116.75, -116.5 , -116.25,
       -116.  ], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -122.0 -121.8 -121.5 ... -116.5 -116.2 -116.0
Attributes:
    units:      degrees_east
    long_name:  longitude
<xarray.DataArray 'latitude' (latitude: 33)>
array([42.  , 41.75, 41.5 , 41.25, 41.  , 40.75, 40.5 , 40.25, 40.  , 39.75,
       39.5 , 39.25, 39.  , 38.75, 38.5 , 38.25, 38.  , 37.75, 37.5 , 37.25,
       37.  , 36.75, 36.5 , 36.25, 36.  , 35.75, 35.5 , 35.25, 35.  , 34.75,
       34.5 , 34.25, 34.  ], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 42.0 41.75 41.5 41.25 ... 34.75 34.5 34.25 34.0
Attributes:
    units:      degrees_north
    long_name:  latitude


In [6]:
trap_data = pd.read_csv("/Volumes/My Book/Synchrony/_data/ll_info_23.csv")

In [7]:
def downsample_df(dataset):
    dataset['t2m'] -= 273.15
    dataset.t2m.attrs['units'] = 'deg C'
    max_daily = dataset.resample(time='D').max(dim='time')
    min_daily = dataset.resample(time='D').min(dim='time')
    mean_daily = dataset.resample(time='D').mean(dim='time')
    
    max_daily = max_daily.rename({'t2m':'max_t2m'})
    min_daily = min_daily.rename({'t2m':'min_t2m'})
    mean_daily = mean_daily.rename({'t2m':'mean_t2m'})
    
    merged_data = xr.merge([max_daily,min_daily,mean_daily])
    
    # faster to do this on the xarray
    merged_data['year'] = merged_data['time'].dt.strftime('%Y')
    merged_data['month'] = merged_data['time'].dt.strftime('%B')
    merged_data['day'] = merged_data['time'].dt.strftime('%d')
    
    df = merged_data.to_dataframe()
    df = df.reset_index()
    
    return(df)

In [10]:
df80 = downsample_df(f80)
df90 = downsample_df(f90)
df00 = downsample_df(f00)
df10 = downsample_df(f10)

In [12]:
df19 = downsample_df(f19)

In [8]:
df23 = downsample_df(f23)

In [11]:
df10

,time,longitude,latitude,expver,max_t2m,min_t2m,mean_t2m,year,month,day
0,2010-01-01,-124.0,52.00,1,-8.546753,-14.145416,-12.234475,2010,January,01
1,2010-01-01,-124.0,52.00,5,NaN,NaN,NaN,2010,January,01
2,2010-01-01,-124.0,51.75,1,-7.023163,-13.069214,-11.059690,2010,January,01
3,2010-01-01,-124.0,51.75,5,NaN,NaN,NaN,2010,January,01
4,2010-01-01,-124.0,51.50,1,-5.778992,-11.122406,-9.014939,2010,January,01
...,...,...,...,...,...,...,...,...,...,...
14826415,2020-12-31,-113.0,42.50,5,-0.697601,-5.296661,-3.657162,2020,December,31
14826416,2020-12-31,-113.0,42.25,1,NaN,NaN,NaN,2020,December,31
14826417,2020-12-31,-113.0,42.25,5,-1.638123,-6.583801,-4.582212,2020,December,31
14826418,2020-12-31,-113.0,42.00,1,NaN,NaN,NaN,2020,December,31


In [ ]:
df_all = pd.concat([df80,df90,df00,df10])

In [13]:
df80.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_80s.csv")
df90.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_90s.csv")
df00.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_00s.csv")
df10.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_10s.csv")
df19.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_19s.csv")

In [10]:
df23.to_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_20s.csv")

### Filtering by traps in the state

In [11]:
df80 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_80s.csv")
df90 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_90s.csv")
df00 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_00s.csv")
df10 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_10s.csv")
df19 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_19s.csv")
df20 = pd.read_csv("/Volumes/My Book/Climate/ERA/downsampled/pnw_20s.csv")

In [12]:
df_all = pd.concat([df80,df90,df00,df10,df19,df20])

In [13]:
#state_df = trap_data.loc[trap_data['State'] == "CO"] 
state_df = trap_data

In [14]:
state_df = state_df.reset_index(drop=True)

In [15]:
trap_lats = state_df.lat
trap_lons = state_df.lon

print(trap_lats)

0       34.452258
1       34.429790
2       33.861750
3       33.816588
4       33.812968
          ...    
1771    45.897575
1772    45.861841
1773    45.826436
1774    45.893359
1775    45.617810
Name: lat, Length: 1776, dtype: float64


In [16]:
min(trap_lats)

32.427858

In [17]:
grid_lon = list(set(df_all.longitude))
grid_lat = list(set(df_all.latitude))

In [18]:
lat_coord = []

for t in range(0,len(trap_lats)):
    lat_coord.append(grid_lat[np.where(abs(trap_lats[t]- grid_lat) == min(abs(trap_lats[t]- grid_lat)))[0][0]])
    
lon_coord = []

for t in range(0,len(trap_lats)):
    lon_coord.append(grid_lon[np.where(abs(trap_lons[t]- grid_lon) == min(abs(trap_lons[t]- grid_lon)))[0][0]])

In [19]:
print(min(lat_coord),max(lat_coord))
print(min(lon_coord),max(lon_coord))

42.0 51.5
-123.75 -109.0


In [20]:
coord_df = pd.DataFrame({'lat' : trap_lats, 'lon' : trap_lons,
                     'lat_coord' : lat_coord, 'lon_coord': lon_coord})

In [21]:
state_df = pd.merge(state_df,coord_df)

In [22]:
state_df["key"] = state_df["lat_coord"].astype(str) + state_df["lon_coord"].astype(str)

In [23]:
state_keys = list(set(state_df.key))

In [24]:
state_keys

['42.0-109.0',
 '42.0-120.75',
 '50.0-119.5',
 '45.25-117.75',
 '45.0-121.5',
 '49.0-119.25',
 '46.25-116.75',
 '44.0-119.0',
 '46.75-115.5',
 '47.75-117.25',
 '48.0-117.75',
 '48.0-117.25',
 '45.0-117.25',
 '45.75-117.5',
 '47.25-116.5',
 '42.5-121.0',
 '44.75-117.0',
 '49.0-119.0',
 '51.25-121.25',
 '48.25-117.75',
 '48.25-120.75',
 '45.25-119.0',
 '48.0-120.5',
 '47.5-121.25',
 '46.0-113.0',
 '47.0-116.25',
 '42.0-120.5',
 '50.75-121.0',
 '43.5-113.5',
 '45.5-118.0',
 '48.5-117.25',
 '50.25-119.25',
 '44.0-115.5',
 '48.75-119.75',
 '44.25-115.5',
 '48.75-118.0',
 '48.25-118.0',
 '45.0-116.0',
 '45.25-114.5',
 '42.0-111.25',
 '49.0-117.5',
 '45.25-118.0',
 '45.0-119.0',
 '48.75-118.75',
 '45.0-114.25',
 '47.0-116.0',
 '48.5-117.5',
 '45.25-116.75',
 '47.0-117.25',
 '48.25-117.0',
 '45.0-118.75',
 '44.0-119.5',
 '42.5-122.25',
 '42.0-123.75',
 '42.0-119.5',
 '45.25-119.5',
 '42.0-121.25',
 '42.0-119.0',
 '45.75-117.0',
 '43.0-116.75',
 '43.75-115.5',
 '43.75-118.75',
 '48.0-120.25',
 

In [25]:
df80["key"] = df80["latitude"].astype(str) + df80["longitude"].astype(str)
df90["key"] = df90["latitude"].astype(str) + df90["longitude"].astype(str)
df00["key"] = df00["latitude"].astype(str) + df00["longitude"].astype(str)
df10["key"] = df10["latitude"].astype(str) + df10["longitude"].astype(str)
df19["key"] = df19["latitude"].astype(str) + df19["longitude"].astype(str)
df20["key"] = df20["latitude"].astype(str) + df20["longitude"].astype(str)


In [26]:
state80 = df80[df80['key'].isin(state_keys)]
state90 = df90[df90['key'].isin(state_keys)]
state00 = df00[df00['key'].isin(state_keys)]
state10 = df10[df10['key'].isin(state_keys)]
state19 = df19[df19['key'].isin(state_keys)]
state20 = df20[df20['key'].isin(state_keys)]

In [27]:
state_all = pd.concat([state80, state90,state00,state10,state19,state20], axis=0).reset_index(drop=True)

In [28]:
state_all

,Unnamed: 0,time,longitude,latitude,max_t2m,min_t2m,mean_t2m,year,month,day,key,expver
0,81,1979-01-01,-123.75,42.00,7.164520,-5.614166,-1.741871,1979,January,1,42.0-123.75,NaN
1,120,1979-01-01,-123.50,42.50,4.168854,-7.914856,-3.556188,1979,January,1,42.5-123.5,NaN
2,121,1979-01-01,-123.50,42.25,5.306213,-7.184387,-2.742933,1979,January,1,42.25-123.5,NaN
3,122,1979-01-01,-123.50,42.00,5.732208,-6.767944,-2.604856,1979,January,1,42.0-123.5,NaN
4,161,1979-01-01,-123.25,42.50,4.196167,-7.588532,-3.364012,1979,January,1,42.5-123.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8480383,5210173,2023-11-25,-109.25,45.00,-5.102448,-17.378280,-13.334290,2023,November,25,45.0-109.25,5.0
8480384,5210196,2023-11-25,-109.25,42.00,NaN,NaN,NaN,2023,November,25,42.0-109.25,1.0
8480385,5210197,2023-11-25,-109.25,42.00,-5.515991,-11.870575,-10.163630,2023,November,25,42.0-109.25,5.0
8480386,5210278,2023-11-25,-109.00,42.00,NaN,NaN,NaN,2023,November,25,42.0-109.0,1.0


In [29]:
state_all.to_csv("/Volumes/My Book/Climate/ERA/downsampled/yearly_aggregates_pnw23.csv")

### Missing 2019 in dataset

In [46]:
f1 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+"AZ"+"_T2M_2009.nc", decode_times = True)
f2 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+"CA"+"_T2M_2009.nc", decode_times = True)
f3 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+"CO"+"_T2M_2009.nc", decode_times = True)
f4 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+"NM"+"_T2M_2009.nc", decode_times = True)

In [47]:
df1 = downsample_df(f1)
df2 = downsample_df(f2)
df3 = downsample_df(f3)
df4 = downsample_df(f4)

In [64]:
df_all = pd.concat([df1,df2,df3,df4])

In [65]:
df_all["key"] = df_all["latitude"].astype(str) + df_all["longitude"].astype(str)

In [86]:
state_all = df_all[df_all['key'].isin(state_keys)]

In [87]:
state_all

,time,longitude,latitude,max_t2m,min_t2m,mean_t2m,year,month,day,key
0,2019-01-01,-112.50,34.50,1.160858,-6.772614,-2.822146,2019,January,01,34.5-112.5
40,2019-01-01,-111.50,34.50,-3.132050,-11.618011,-6.575296,2019,January,01,34.5-111.5
50,2019-01-01,-111.25,34.50,-4.851440,-12.973724,-7.821532,2019,January,01,34.5-111.25
61,2019-01-01,-111.00,34.25,-3.664551,-10.367188,-6.254099,2019,January,01,34.25-111.0
63,2019-01-01,-111.00,33.75,0.739258,-2.963226,-0.797151,2019,January,01,33.75-111.0
...,...,...,...,...,...,...,...,...,...,...
46701,2019-12-31,-105.75,33.00,5.291351,-6.083405,-1.195306,2019,December,31,33.0-105.75
46702,2019-12-31,-105.75,32.75,6.814545,-4.245819,0.330424,2019,December,31,32.75-105.75
46704,2019-12-31,-105.50,36.25,0.392426,-18.785370,-11.065396,2019,December,31,36.25-105.5
46717,2019-12-31,-105.50,33.00,5.076447,-6.894257,-1.892649,2019,December,31,33.0-105.5


In [88]:
min(state_all.latitude)

32.5

In [89]:
state_all = state_all.reset_index(drop=True)

In [90]:
state_all.to_csv("/Volumes/My Book/Climate/ERA/downsampled/yearly_aggregates_2009.csv")